# eCommerce Product Recommendation - Part 2

# 1. Load Raw Data to Colab

In [ ]:
import pandas as pd
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from collections import OrderedDict

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
cd /content/drive/MyDrive/Laioffer/csv_files/RecSys_eComm/

/content/drive/MyDrive/Laioffer/csv_files/RecSys_eComm


In [ ]:
ls

aisles.csv       model_all_data.csv  order_products_prior.csv  orders.csv
departments.csv  model.png           order_products_train.csv  products.csv


In [ ]:
aisles = pd.read_csv('/content/drive/MyDrive/RecSys_eComm/aisles.csv')
departments = pd.read_csv('/content/drive/MyDrive/RecSys_eComm/departments.csv')
order_products_prior = pd.read_csv('/content/drive/MyDrive/RecSys_eComm/order_products_prior.csv')
order_products_train = pd.read_csv('/content/drive/MyDrive/RecSys_eComm/order_products_train.csv')
orders = pd.read_csv('/content/drive/MyDrive/RecSys_eComm/orders.csv')
products = pd.read_csv('/content/drive/MyDrive/RecSys_eComm/products.csv')

# 1. Construct Model Label

思考: 为什么不能直接使用`reordered`column中的数据作为model label？

reordered: 1 if this product has been ordered by this user in the past, 0 otherwise

- 买过1次 or 多次的，均为1. 从未购买过为0.
- 我们需要在1里，找出购买过2次 or 2次以上的客户。


思考：为什么我们需要使用both prior & train datasets

### 答: 我们最终目标是预测未来,但是没有未来数据,所以我门要在过去某个时间节点砍一刀,吧这个时间节点之前的一段时间作为Train, 之后的时间作为test




### double click this chunk to view the relationship! 
这是我们自己设定的label! 
如何split prior和train? 公司不同组定义不同, 如可以是prior过去的第4-9个月, train为最近1-3个月

      Ex. 2021                              Ex.2022.         predict
[------prior--------------] @ [---------------train-------] ---------> future 
                           Tiff
      0  Never                         0 没复购
      1  复购                           1 复购

train label 在这里等同于为未来 也是我们要predict的label, 如果prior是0 无论未来买不买, 都不算复购, 因为我们对于复购的定义就是在prior阶段买了产品,再在train阶段再买才算复购.

prior 0 +train 0 = label 0 2021没买 无论2022买不买 直接label为0
prior 0 + train 1 = label 0 2021没买 无论2022买不买 直接label为0
prior 1 + train 0 = label 0 如果2021买了, 2022没买, 哪label是0
prior 1 + train 1 = label 1 如果2021,2022都买了 才算label 1, 因为train是我们要预测的data,如果train是1 才代表真的复购了


### Now we start to add the label (Step 1 - 5)

In [ ]:
#Step1: get the all information of the prior dataset because orders contains addtional features we needed for prior
prior_all_info = order_products_prior.merge(orders, on = 'order_id')
prior_all_info.head(3)

,order_id,product_id,add_to_cart_order,reordered,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order
0,12,30597,1,1,152610,prior,22,6,8,10.0
1,12,15221,2,1,152610,prior,22,6,8,10.0
2,12,43772,3,1,152610,prior,22,6,8,10.0


In [ ]:
# Step2: get the all information of the train dataset because orders contains addtional features we needed for prior
train_all_info = order_products_train.merge(orders, on = 'order_id')
train_all_info.head(3)

,order_id,product_id,add_to_cart_order,reordered,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order
0,1077,13176,1,1,173934,train,11,6,9,10.0
1,1077,39922,2,1,173934,train,11,6,9,10.0
2,1077,5258,3,1,173934,train,11,6,9,10.0


In [ ]:
#Step 3 get the unique user_id set of training data and use it to filter the prior dataset
#reason: we are predicting label in the training data, so we only have to keep user_id present in the training data
user_id_set = set(train_all_info.user_id)
filtered_prior_all_info = prior_all_info.loc[prior_all_info.user_id.isin(user_id_set)]

In [ ]:
# Step4 Create unique_key as we want to combine user and product together for easier filtering in the later part
filtered_prior_all_info['unique_key'] = filtered_prior_all_info['user_id'].astype('str') + '_' + filtered_prior_all_info['product_id'].astype('str')
train_all_info['unique_key'] = train_all_info['user_id'].astype('str') + '_' + train_all_info['product_id'].astype('str')
filtered_prior_all_info.head(3) 

,order_id,product_id,add_to_cart_order,reordered,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order,unique_key
0,12,30597,1,1,152610,prior,22,6,8,10.0,152610_30597
1,12,15221,2,1,152610,prior,22,6,8,10.0,152610_15221
2,12,43772,3,1,152610,prior,22,6,8,10.0,152610_43772


In [ ]:
#Step5 drop duplicate in filtered_prior_all_info
#purpose: we have to make sure only one unique_key (user + product combination )present in dataset for later ML model
print(filtered_prior_all_info.shape)
filtered_prior_all_info.nunique() #we find unique_key are around 330,000 while dataset has nearly 800,000 data

(793924, 11)


order_id                   77683
product_id                 28927
add_to_cart_order             92
reordered                      2
user_id                     5000
eval_set                       1
order_number                  99
order_dow                      7
order_hour_of_day             24
days_since_prior_order        31
unique_key                329806
dtype: int64

In [ ]:
#step 5 continued
#how do we drop duplicate? 
#if unique_key present once, we keep it;
#if unique_key present more than once, meaning this user purphase this product more than once during the prior period, then there must be a reorder = 1, then we need to keep the last reorder = 1 (lastest data)
filtered_prior_all_info.unique_key.value_counts() # Here we find some unique_key presents multiple times

164055_4210     86
95967_33198     85
164055_8195     80
154977_18531    79
154977_19660    78
                ..
200245_17834     1
200245_31315     1
92926_9339       1
92926_32330      1
186579_13852     1
Name: unique_key, Length: 329806, dtype: int64

In [ ]:
#step5 continued (This step main aim is to delete duplicate unique_values and only keep the last reorder = 1 of them)

#create unique key set appears more than once
unique_key_table = filtered_prior_all_info.unique_key.value_counts().to_frame()
unique_key_more_than_one_set = set(unique_key_table.loc[unique_key_table.unique_key > 1].index)

#we delete data that has uniquekey more than once and reorder = 0
index_drop = filtered_prior_all_info[(filtered_prior_all_info.unique_key.isin(unique_key_more_than_one_set)) & (filtered_prior_all_info.reordered == 0) ].index
filtered_prior_all_info.drop(index_drop , inplace=True)

#keep filtering: only keep the last depulicated unique_value with reorder = 1
filtered_prior_all_info.drop_duplicates(subset = ['unique_key'],keep='last',inplace=True,ignore_index=True)
filtered_prior_all_info.shape #after filtering we have 330,000 data

(329806, 11)

In [ ]:
# Step 5 adding label -- base on rule discussed above: only train and prior both are 1 then label is 1
unique_key_train = set(train_all_info.loc[train_all_info.reordered == 1].unique_key)

filtered_prior_all_info['label'] = 0
filtered_prior_all_info.loc[(filtered_prior_all_info.reordered == 1) & (filtered_prior_all_info.unique_key.isin(unique_key_train)), 'label'] = 1 
filtered_prior_all_info.label.value_counts()

0    306866
1     22940
Name: label, dtype: int64

In [ ]:
# Step 6 keep useful feature
model_all_data = filtered_prior_all_info[['user_id',	'product_id',	'order_number',	'order_dow',	'order_hour_of_day',	'days_since_prior_order',	'unique_key',	'label']]

In [ ]:
model_all_data.shape

(329806, 8)

# 2. Construct Model Features (3 different group) 

### This step we totally create 20 features using various rules. From 8 to 28.

思考：为什么我们需要单独使用prior dataset --> 原因: feature需来自于historical dataset

2.1. Feature Group 1: user-product activity features （用户买了什么东西）

In [ ]:
user_product_features = ['user_product__total_orders',
                         'user_product__add_to_cart_order_mean',
                         'user_product__reordered_mean',
                         'user_product__most_dow',
                         'user_product__most_hod']

In [ ]:
prior_all_info.head()

,order_id,product_id,add_to_cart_order,reordered,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order
0,12,30597,1,1,152610,prior,22,6,8,10.0
1,12,15221,2,1,152610,prior,22,6,8,10.0
2,12,43772,3,1,152610,prior,22,6,8,10.0
3,12,37886,4,1,152610,prior,22,6,8,10.0
4,12,37215,5,0,152610,prior,22,6,8,10.0


In [ ]:
# 注意: 这个block的程序执行时间约为2mins
# The only difference between dict() and OrderedDict() is that: OrderedDict preserves the order in which the keys are inserted. 
# Use OrderedDict to maintain the order of the columns ('count', 'mean', 'mean' ... ) for the dataframe that is returned (方便我们后面加刚刚定义的label,确保顺序一致)
df_user_product_features = (prior_all_info.groupby(['product_id','user_id'],as_index=False) 
                                           .agg(OrderedDict(              
                                                   [('order_id','count'), # 该用户对该商品的购买次数
                                                    ('add_to_cart_order','mean'), # 该用户对该商品的加入购物车顺序 - 均值
                                                    ('reordered', 'mean'), #该用户对该商品的复购率
                                                    ('order_dow', (lambda x: x.mode()[0])), #该用户购买该商品，最多的dow
                                                    ('order_hour_of_day', (lambda x: x.mode()[0])), #该用户购买该商品，最多的 hod
                                                    ])))
df_user_product_features.columns = ['product_id', 'user_id'] + user_product_features

In [ ]:
model_all_data = model_all_data.merge(df_user_product_features, on = ['user_id', 'product_id'])
model_all_data.head(3)

,user_id,product_id,order_number,order_dow,order_hour_of_day,days_since_prior_order,unique_key,label,user_product__total_orders,user_product__add_to_cart_order_mean,user_product__reordered_mean,user_product__most_dow,user_product__most_hod
0,152610,43772,22,6,8,10.0,152610_43772,0,2,3.0,0.5,5,8
1,152610,37886,22,6,8,10.0,152610_37886,0,2,3.5,0.5,6,8
2,152610,37215,22,6,8,10.0,152610_37215,0,1,5.0,0.0,6,8


2.2. Feature Group 2: product features （间接features，如何描述这些products）

product features分为两种，一种是根据所有用户的行为总结出的product features, 另一种是根据product本身信息得到的features。

In [ ]:
product_features = ['product__total_orders',
                     'product__add_to_cart_order_mean',
                     'product__total_users',
                     'product__reordered_mean',
                     'product__most_dow',
                     'product__most_hod',
                     'product__days_since_prior_order_mean'
                     ]

In [ ]:
df_product_features = (prior_all_info.groupby(['product_id'],as_index=False)
                                           .agg(OrderedDict(
                                                   [('order_id','nunique'), # 该产品被所有用户购买的次数
                                                    ('add_to_cart_order','mean'), #该产品被所有用户添加到购物车的顺序 - 平均值
                                                    ('user_id', 'nunique'), # 所有购买该产品的人数
                                                    ('reordered', 'mean'), #该产品被所有用户复购的频率 (也可以加总和，方差)
                                                    ('order_dow', (lambda x: x.mode()[0])), #该产品被购买最多的dow #
                                                    ('order_hour_of_day', (lambda x: x.mode()[0])), #该产品被购买最多的dow
                                                    ('days_since_prior_order', 'mean') #该商品距离上次被购买的均值
                                                    ])))
df_product_features.columns = ['product_id'] + product_features

In [ ]:
model_all_data = model_all_data.merge(df_product_features, on = ['product_id'])
model_all_data = model_all_data.merge(products[['product_id','aisle_id', 'department_id']], 
                                      on = ['product_id'])
model_all_data.rename(columns={'aisle_id': 'product__aisle_id', 'department_id': 'product__department_id'}, inplace=True)
model_all_data.head(3)

,user_id,product_id,order_number,order_dow,order_hour_of_day,days_since_prior_order,unique_key,label,user_product__total_orders,user_product__add_to_cart_order_mean,...,user_product__most_hod,product__total_orders,product__add_to_cart_order_mean,product__total_users,product__reordered_mean,product__most_dow,product__most_hod,product__days_since_prior_order_mean,product__aisle_id,product__department_id
0,152610,43772,22,6,8,10.0,152610_43772,0,2,3.0,...,8,508,9.096457,171,0.663386,1,12,12.008565,83,4
1,38275,43772,13,3,15,24.0,38275_43772,0,2,4.0,...,14,508,9.096457,171,0.663386,1,12,12.008565,83,4
2,111084,43772,5,2,9,22.0,111084_43772,0,1,21.0,...,9,508,9.096457,171,0.663386,1,12,12.008565,83,4


2.3. Feature Group 3: user features

In [ ]:
user_features = ['user__order_count',
                  'user__product_count',
                  'user__days_since_prior_order_mean',
                  'user__reordered_mean',
                  'user__most_dow',
                  'user__most_hod',
                  ]

In [ ]:
df_user_features = (prior_all_info.groupby(['user_id'],as_index=False)
                                           .agg(OrderedDict(
                                                   [('order_id','nunique'), # 该用户的总订单数
                                                    ('product_id','count'), #该用户购买的全部商品总数
                                                    ('days_since_prior_order','mean'), #该用户订单平均间隔时间
                                                    ('reordered', 'mean'), #该用户的总复购率
                                                    ('order_dow', (lambda x: x.mode()[0])), #该用户下单最多的dow
                                                    ('order_hour_of_day', (lambda x: x.mode()[0])), #该用户下单最多的dow
                                                    ])))
df_user_features.columns = ['user_id'] + user_features

In [ ]:
model_all_data = model_all_data.merge(df_user_features, on = ['user_id'])

In [ ]:
model_all_data.shape 

(329806, 28)

# 3. Save feature data to google drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
cd /content/drive/MyDrive/RecSys_eComm

/content/drive/MyDrive/RecSys_eComm


In [ ]:
# Save the model_all_data dataframe as csv file to the file path we selected above.
model_all_data.to_csv('model_all_data.csv', index=False)

#### normally interview stop at here, modeling part is not as important as previous part